In [1]:
import requests
import json
import config
import openai
import tqdm
import html
from datetime import datetime
from story_feedback import WorkItem, get_work_item

ADO_URL = f"https://dev.azure.com/"
ADO_URL += f"{config.DEVOPS_ORG}/" 
ADO_URL += f"_apis/wit/wiql"

api_version_param = {"api-version":config.API_VERSION}

class QueryExecutionError(Exception):
    """Exception raised for errors during query execution."""
    pass

def run_query(query, params={}):
    query_params = api_version_param | params

    payload = json.dumps({"query":query})
    response = requests.post(ADO_URL, headers=config.PRIMARY_HEADERS, data=payload, params=query_params)
    if response.status_code != 200:
        raise QueryExecutionError(f"Failed to execute query. Error: {response.json()}")

    return response.json() 

In [28]:
# Run a query to get the workitems from a particular project
work_items_by_project_query = """
SELECT
    [System.Id],
    [System.WorkItemType],
    [System.Title],
    [System.AssignedTo],
    [System.State],
    [System.Tags]
FROM workitems
WHERE
    [System.TeamProject] = '{project_name}'
    AND [System.WorkItemType] = 'User Story'
ORDER BY [System.Id]
"""

projects = ["F23 - P1", 'F23-P3', 'F23-P4', 'FinderSpa.com', 'PetPulse (F23-P2)']

def get_work_items_by_project(project_name):
    wiql_query = work_items_by_project_query.format(project_name=project_name)
    return run_query(query=wiql_query)['workItems']

work_items = get_work_items_by_project('Sample Project')
work_item_urls = [w.get("id") for w in work_items]

# Gather the descriptions and story points given the urls or ids

[7, 22, 25, 26, 31, 32, 230, 231, 308, 309, 311, 577, 689]


In [4]:
wi = get_work_item(7)
wi

In [15]:
def get_chat_feedback(work_items):
    '''
    Go to ChatGPT and ask for feedback on the user story.
    
    :returns: response from ChatGPT
    '''
    REQUEST = """The following are user stories and the story points associated with them.
                Consider them all, on the whole.  Do any of them standout as having too few
                or too many story points associated with them?  Are they consistent in sizing?
                Which ones, if any should be adjusted?  When you respond start with either GOOD: or NEEDS WORK:
    """
    conversation= [{"role":"system", "content":REQUEST},]
    for wi in work_items:
        conversation += [  
               {"role":"user", "content":wi.description},
               {"role":"user", "content":wi.acceptance_criteria},
               {"role":"user", "content":f"story points {wi.story_points}"}]
    
    response = openai.ChatCompletion.create(model='gpt-3.5-turbo',messages=conversation)
    message = response['choices'][0]['message']['content']
    return message

#wi_list = [get_work_item(i) for i in [7, 22, 25, 26, 31, 32, 230, 231, 308, 309, 311, 577, 689]]
#get_chat_feedback()


In [20]:
get_chat_feedback([wi_list[3]])

'GOOD: The story points for this user story seem appropriate. It involves the ability to view the release log and have it automatically updated, which could require some development effort. A story point of 8.0 indicates that it is a medium-sized user story.'

# Update just the quality field from the feedback already provided

In [16]:
# Enable the autoreload extension
%load_ext autoreload

# Set autoreload mode to automatically reload all modules (mode 2)
%autoreload 2

import requests
import json
import config
from openai import OpenAI
from story_feedback import WorkItem, get_work_item, determine_quality, get_last_comment
import logging
logging.basicConfig(level=logging.INFO)

INFO = logging.INFO
DEBUG = logging.DEBUG
ERROR = logging.ERROR

client = OpenAI(api_key=config.OPEN_API_KEY)

# Get the open user stories which have been AI reviewed
def fetch_ai_reviewed_user_stories():
    url = f"https://dev.azure.com/"
    url += f"{config.DEVOPS_ORG}/" 
    url += f"_apis/wit/wiql"

    params = {"api-version":config.API_VERSION}
    query = """
            SELECT[System.Id] FROM workitems
            WHERE[System.WorkItemType] = 'User Story'
            AND [Custom.AIReviewed] = 'True' AND [System.ChangedDate] >= @Today - 60
            """
    payload = json.dumps({"query": query
    })
    # Get the list of work items
    response = requests.post(url, headers=config.PRIMARY_HEADERS, data=payload, params=params)
    if response.status_code != 200:
        ERROR(f"Failed to fetch user stories. Error: {response}")
        return []
    work_items = response.json()['workItems']
    logging.debug(f'Found {len(work_items)} user stories to review.')    
    return work_items

def update_quality(work_item_id, quality):
    url = f"https://dev.azure.com/{config.DEVOPS_ORG}/_apis/wit/workitems/{work_item_id}"

    headers = {
        'Content-Type': 'application/json-patch+json',
        'Authorization': f'Basic {config.DEVOPS_AUTH_TOKEN}',
    }
    params = {"api-version":config.API_VERSION}
    
    # Prepare the payload
    payload = [
        {
            "op": "add",
            "path": "/fields/Custom.Quality",
            "value": quality
        }
    ]
    response = requests.request("PATCH", url, headers=headers, data=json.dumps(payload), params=params)

    if response.status_code == 200:
        print("Work item updated successfully.")
    else:
        print(f"Failed to update work item. Status code: {response.status_code}, Error: {response.text}")

# Get the work items from the list of work item ids
work_items = []
for wi in fetch_ai_reviewed_user_stories():
    workitem_id = wi['id']
    workitem_id = 1436
    workitem_comments = get_last_comment(workitem_id)
    quality = determine_quality(workitem_comments)
    if len(quality)>0:
        update_quality(wi['id'], quality)
    
    
    

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Work item updated successfully.


KeyboardInterrupt: 

In [9]:
headers = {
        'Content-Type': 'application/json-patch+json',
        'Authorization': f'Basic {config.DEVOPS_AUTH_TOKEN}',
    }
params = {"api-version":config.API_VERSION}

# Fetch work item revisions
def fetch_work_item_revisions(id):
    url = f"https://dev.azure.com/{config.DEVOPS_ORG}/_apis/wit/workItems/{id}/revisions?api-version=7.1-preview.3"
    response = requests.get(url, headers=headers)
    return response.json()

# Fetch work item comments
def fetch_work_item_comments(id):
    url = f"https://dev.azure.com/{config.DEVOPS_ORG}/_apis/wit/workItems/{id}/comments"
    response = requests.get(url, headers=headers)
    return response.json()

# Get the last comment
comments = fetch_work_item_comments(1489)
if comments.get('comments'):
    last_comment = comments['comments'][-1]['text']  # Get the last comment's text
else:
    last_comment = "No comments available"

# Use the history as needed
print(last_comment)

Overall Quality: Good<br><br>- Independent: Good<br>  - The user story seems to be self-contained and does not rely on other stories.<br><br>- Negotiable: Good<br>  - The user story leaves room for discussion on how the FAQ entry functionality will be implemented, which is a good quality.<br><br>- Valuable: Good<br>  - Adding new FAQ entries to keep the information up-to-date is valuable for users and administrators.<br><br>- Estimable: Good<br>  - The tasks involved in creating and submitting new FAQ entries, ensuring they appear correctly, and other specified requirements can be estimated for effort.<br><br>- Small: Good<br>  - The scope of adding new FAQ entries and ensuring their correctness is small enough to be completed within a sprint.<br><br>- Testable: Good<br>  - The acceptance criteria provided are clear, specific, and testable, ensuring that the story's completion can be verified.<br><br>Acceptance Criteria Quality: Great<br>  - The acceptance criteria provided are specifi